# Advanced Retrieval with LangChain

In the following notebook, we'll explore various methods of advanced retrieval using LangChain!

We'll touch on:

- Naive Retrieval
- Best-Matching 25 (BM25)
- Multi-Query Retrieval
- Parent-Document Retrieval
- Contextual Compression (a.k.a. Rerank)
- Ensemble Retrieval
- Semantic chunking

We'll also discuss how these methods impact performance on our set of documents with a simple RAG chain.

There will be two breakout rooms:

- 🤝 Breakout Room Part #1
  - Task 1: Getting Dependencies!
  - Task 2: Data Collection and Preparation
  - Task 3: Setting Up QDrant!
  - Task 4-10: Retrieval Strategies
- 🤝 Breakout Room Part #2
  - Activity: Evaluate with Ragas

# 🤝 Breakout Room Part #1

## Task 1: Getting Dependencies!

We're going to need a few specific LangChain community packages, like OpenAI (for our [LLM](https://platform.openai.com/docs/models) and [Embedding Model](https://platform.openai.com/docs/guides/embeddings)) and Cohere (for our [Reranker](https://cohere.com/rerank)).

> You do not need to run the following cells if you are running this notebook locally. 

In [1]:
#!pip install -qU langchain langchain-openai langchain-cohere rank_bm25

We're also going to be leveraging [Qdrant's](https://qdrant.tech/documentation/frameworks/langchain/) (pronounced "Quadrant") VectorDB in "memory" mode (so we can leverage it locally in our colab environment).

In [ ]:
#!pip install -qU qdrant-client

We'll also provide our OpenAI key, as well as our Cohere API key.

In [10]:
import os
import getpass


os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

In [11]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

## Task 2: Data Collection and Preparation

We'll be using some reviews from the 4 movies in the John Wick franchise today to explore the different retrieval strategies.

These were obtained from IMDB, and are available in the [AIM Data Repository](https://github.com/AI-Maker-Space/DataRepository).

### Data Collection

We can simply `wget` these from GitHub.

You could use any review data you wanted in this step - just be careful to make sure your metadata is aligned with your choice.

In [12]:
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv -O john_wick_1.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv -O john_wick_2.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw3.csv -O john_wick_3.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw4.csv -O john_wick_4.csv

--2025-03-06 21:03:19--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19628 (19K) [text/plain]
Saving to: ‘john_wick_1.csv’

john_wick_1.csv     100%[===================>]  19.17K  --.-KB/s    in 0.002s  

2025-03-06 21:03:19 (11.7 MB/s) - ‘john_wick_1.csv’ saved [19628/19628]

--2025-03-06 21:03:19--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14747 (14K) [text/plain]
Sa

### Data Preparation

We want to make sure all our documents have the relevant metadata for the various retrieval strategies we're going to be applying today.

- Self-Query: Wants as much metadata as we can provide
- Time-weighted: Wants temporal data

> NOTE: While we're creating a temporal relationship based on when these movies came out for illustrative purposes, it needs to be clear that the "time-weighting" in the Time-weighted Retriever is based on when the document was *accessed* last - not when it was created.

In [13]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

documents = []

for i in range(1, 5):
  loader = CSVLoader(
      file_path=f"john_wick_{i}.csv",
      metadata_columns=["Review_Date", "Review_Title", "Review_Url", "Author", "Rating"]
  )

  movie_docs = loader.load()
  for doc in movie_docs:

    # Add the "Movie Title" (John Wick 1, 2, ...)
    doc.metadata["Movie_Title"] = f"John Wick {i}"

    # convert "Rating" to an `int`, if no rating is provided - assume 0 rating
    doc.metadata["Rating"] = int(doc.metadata["Rating"]) if doc.metadata["Rating"] else 0

    # newer movies have a more recent "last_accessed_at"
    doc.metadata["last_accessed_at"] = datetime.now() - timedelta(days=4-i)

  documents.extend(movie_docs)

Let's look at an example document to see if everything worked as expected!

In [14]:
documents[0]

Document(metadata={'source': 'john_wick_1.csv', 'row': 0, 'Review_Date': '6 May 2015', 'Review_Title': ' Kinetic, concise, and stylish; John Wick kicks ass.\n', 'Review_Url': '/review/rw3233896/?ref_=tt_urv', 'Author': 'lnvicta', 'Rating': 8, 'Movie_Title': 'John Wick 1', 'last_accessed_at': datetime.datetime(2025, 3, 3, 21, 3, 36, 392645)}, page_content=": 0\nReview: The best way I can describe John Wick is to picture Taken but instead of Liam Neeson it's Keanu Reeves and instead of his daughter it's his dog. That's essentially the plot of the movie. John Wick (Reeves) is out to seek revenge on the people who took something he loved from him. It's a beautifully simple premise for an action movie - when action movies get convoluted, they get bad i.e. A Good Day to Die Hard. John Wick gives the viewers what they want: Awesome action, stylish stunts, kinetic chaos, and a relatable hero to tie it all together. John Wick succeeds in its simplicity.")

## Task 3: Setting up QDrant!

Now that we have our documents, let's create a QDrant VectorStore with the collection name "JohnWick".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

> NOTE: We'll be creating additional vectorstores where necessary, but this pattern is still extremely useful.

In [15]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWick"
)

## Task 4: Naive RAG Chain

Since we're focusing on the "R" in RAG today - we'll create our Retriever first.

### R - Retrieval

This naive retriever will simply look at each review as a document, and use cosine-similarity to fetch the 10 most relevant documents.

> NOTE: We're choosing `10` as our `k` here to provide enough documents for our reranking process later

In [16]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

### A - Augmented

We're going to go with a standard prompt for our simple RAG chain today! Nothing fancy here, we want this to mostly be about the Retrieval process.

In [17]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### G - Generation

We're going to leverage `gpt-3.5-turbo` as our LLM today, as - again - we want this to largely be about the Retrieval process.

In [18]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI()

### LCEL RAG Chain

We're going to use LCEL to construct our chain.

> NOTE: This chain will be exactly the same across the various examples with the exception of our Retriever!

In [19]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's see how this simple chain does on a few different prompts.

> NOTE: You might think that we've cherry picked prompts that showcase the individual skill of each of the retrieval strategies - you'd be correct!

In [20]:
naive_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"Yes, people generally liked John Wick. The reviews praise Keanu Reeves' performance, the action sequences, and the overall entertainment value of the film."

In [21]:
naive_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10 for the movie "John Wick 3". Here is the URL to that review: \'/review/rw4854296/?ref_=tt_urv\'.'

In [22]:
naive_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, an ex-hitman comes out of retirement to seek vengeance on the gangsters who killed his dog and took everything from him. It leads to a series of violent encounters and battles as he faces off against various enemies.'

Overall, this is not bad! Let's see if we can make it better!

## Task 5: Best-Matching 25 (BM25) Retriever

Taking a step back in time - [BM25](https://www.nowpublishers.com/article/Details/INR-019) is based on [Bag-Of-Words](https://en.wikipedia.org/wiki/Bag-of-words_model) which is a sparse representation of text.

In essence, it's a way to compare how similar two pieces of text are based on the words they both contain.

This retriever is very straightforward to set-up! Let's see it happen down below!


In [23]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(documents)

We'll construct the same chain - only changing the retriever.

In [24]:
bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at the responses!

In [23]:
bm25_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"Opinions on the John Wick movies vary. Some people enjoyed the action and style of the films, while others found them lacking in plot and substance. So, it's not accurate to say that people generally liked John Wick, as there are differing viewpoints on the movies."

In [24]:
bm25_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'No reviews have a rating of 10 in the context provided.'

In [25]:
bm25_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, the main character John Wick is a retired hitman seeking vengeance for the death of his dog, which was a final gift from his late wife. This sets off a series of intense action sequences involving assassins, gunfights, and hand-to-hand combat. The story follows John Wick as he navigates through the criminal underworld to exact his revenge.'

It's not clear that this is better or worse - but the `I don't know` isn't great!

## Task 6: Contextual Compression (Using Reranking)

Contextual Compression is a fairly straightforward idea: We want to "compress" our retrieved context into just the most useful bits.

There are a few ways we can achieve this - but we're going to look at a specific example called reranking.

The basic idea here is this:

- We retrieve lots of documents that are very likely related to our query vector
- We "compress" those documents into a smaller set of *more* related documents using a reranking algorithm.

We'll be leveraging Cohere's Rerank model for our reranker today!

All we need to do is the following:

- Create a basic retriever
- Create a compressor (reranker, in this case)

That's it!

Let's see it in the code below!

In [25]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-english-v3.0")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

In [26]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [27]:
contextual_compression_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Yes, people generally liked John Wick based on the positive reviews and high ratings given by reviewers.'

In [28]:
contextual_compression_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10. Here is the URL to that review:\n- Review URL: /review/rw4854296/?ref_=tt_urv'

In [29]:
contextual_compression_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In John Wick, after resolving his issues with the Russian mafia, John Wick returns home. However, mobster Santino D'Antonio visits him, shows Wick's marker, and asks for help. Wick refuses, so Santino blows up his house. Wick later meets Winston at the Continental hotel, who tells him he must honor the marker. Santino then asks Wick to kill his sister in Rome, and when Wick does so, Santino puts a seven-million dollar contract on him, leading to professional killers coming after him. Ultimately, Wick promises to kill Santino since he is no longer protected by the marker."

We'll need to rely on something like Ragas to help us get a better sense of how this is performing overall - but it "feels" better!

## Task 7: Multi-Query Retriever

Typically in RAG we have a single query - the one provided by the user.

What if we had....more than one query!

In essence, a Multi-Query Retriever works by:

1. Taking the original user query and creating `n` number of new user queries using an LLM.
2. Retrieving documents for each query.
3. Using all unique retrieved documents as context

So, how is it to set-up? Not bad! Let's see it down below!



Let's create our chain again, and see how this does!

In [30]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
)

In [31]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [32]:
multi_query_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Yes, people generally liked John Wick based on the positive reviews provided in the context.'

In [33]:
multi_query_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'No reviews have a rating of 10.'

In [34]:
multi_query_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'John Wick is a retired assassin who comes out of retirement in both the first and second movies to seek revenge and deal with various challenges while navigating the criminal underworld. In the first movie, he seeks revenge for the death of his dog and the theft of his car by taking on the Russian Mafia. In the second movie, he is forced back into the world of assassins to honor a marker and faces challenges in different locations while dealing with assassins trying to kill him.'

## Task 8: Parent Document Retriever

A "small-to-big" strategy - the Parent Document Retriever works based on a simple strategy:

1. Each un-split "document" will be designated as a "parent document" (You could use larger chunks of document as well, but our data format allows us to consider the overall document as the parent chunk)
2. Store those "parent documents" in a memory store (not a VectorStore)
3. We will chunk each of those documents into smaller documents, and associate them with their respective parents, and store those in a VectorStore. We'll call those "child chunks".
4. When we query our Retriever, we will do a similarity search comparing our query vector to the "child chunks".
5. Instead of returning the "child chunks", we'll return their associated "parent chunks".

Okay, maybe that was a few steps - but the basic idea is this:

- Search for small documents
- Return big documents

The intuition is that we're likely to find the most relevant information by limiting the amount of semantic information that is encoded in each embedding vector - but we're likely to miss relevant surrounding context if we only use that information.

Let's start by creating our "parent documents" and defining a `RecursiveCharacterTextSplitter`.

In [35]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_docs = documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

We'll need to set up a new QDrant vectorstore - and we'll use another useful pattern to do so!

> NOTE: We are manually defining our embedding dimension, you'll need to change this if you're using a different embedding model.

In [36]:
client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="full_documents",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = Qdrant(
    collection_name="full_documents", embeddings=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

/var/folders/4x/mgh9b4ps5tn1vt9ylv195tx40000gn/T/ipykernel_10415/3574430551.py:8: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  parent_document_vectorstore = Qdrant(


Now we can create our `InMemoryStore` that will hold our "parent documents" - and build our retriever!

In [37]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore = parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

By default, this is empty as we haven't added any documents - let's add some now!

In [38]:
parent_document_retriever.add_documents(parent_docs, ids=None)

We'll create the same chain we did before - but substitute our new `parent_document_retriever`.

In [39]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's give it a whirl!

In [40]:
parent_document_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"People's opinions on John Wick seem to vary. Some really enjoy the series and consider it a wild ride, while others have strong negative opinions about it."

In [41]:
parent_document_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10 for the movie "John Wick 3". Here is the URL to that review: "/review/rw4854296/?ref_=tt_urv".'

In [42]:
parent_document_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, the main character, played by Keanu Reeves, is a retired assassin who comes out of retirement after someone kills his dog and steals his car. He is forced back into the world of assassins to settle old debts and faces off against numerous adversaries in Italy, Canada, and Manhattan. Ultimately, he is involved in a lot of carnage and kills a large number of people throughout the movie.'

Overall, the performance *seems* largely the same. We can leverage a tool like [Ragas]() to more effectively answer the question about the performance.

## Task 9: Ensemble Retriever

In brief, an Ensemble Retriever simply takes 2, or more, retrievers and combines their retrieved documents based on a rank-fusion algorithm.

In this case - we're using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

Setting it up is as easy as providing a list of our desired retrievers - and the weights for each retriever.

In [43]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

We'll pack *all* of these retrievers together in an ensemble.

In [44]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at our results!

In [45]:
ensemble_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"People generally liked John Wick as it was seen as a slick, action-packed film with impressive choreography, stylish stunts, and kinetic chaos. The reviews highlighted Keanu Reeves' performance, the well-orchestrated action sequences, and the overall entertainment value of the movie. The positive sentiments expressed in the reviews indicate that audiences enjoyed the film."

In [47]:
ensemble_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10 for the movie "John Wick 3". Here is the URL to that review:\nhttps://www.imdb.com/review/rw4854296/?ref_=tt_urv'

In [48]:
ensemble_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, the story revolves around an ex-hitman seeking vengeance after gangsters kill his dog and steal his car, leading him to unleash destruction on those who try to stop him. The movie features intense action, suspense, and revenge themes.'

## Task 10: Semantic Chunking

While this is not a retrieval method - it *is* an effective way of increasing retrieval performance on corpora that have clean semantic breaks in them.

Essentially, Semantic Chunking is implemented by:

1. Embedding all sentences in the corpus.
2. Combining or splitting sequences of sentences based on their semantic similarity based on a number of [possible thresholding methods](https://python.langchain.com/docs/how_to/semantic-chunker/):
  - `percentile`
  - `standard_deviation`
  - `interquartile`
  - `gradient`
3. Each sequence of related sentences is kept as a document!

Let's see how to implement this!

> NOTE: You do not need to run this cell if you're running this locally

In [ ]:
#!pip install -qU langchain_experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.1/208.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00


We'll use the `percentile` thresholding method for this example which will:

Calculate all distances between sentences, and then break apart sequences of setences that exceed a given percentile among all distances.

In [46]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

Now we can split our documents.

In [47]:
semantic_documents = semantic_chunker.split_documents(documents)

Let's create a new vector store.

In [48]:
semantic_vectorstore = Qdrant.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWickSemantic"
)

We'll use naive retrieval for this example.

In [49]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

Finally we can create our classic chain!

In [50]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

And view the results!

In [52]:
semantic_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Based on the reviews provided, John Wick seems to have been generally liked by viewers.'

In [53]:
semantic_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10 for the movie "John Wick 3". Here is the URL to that review: /review/rw4854296/?ref_=tt_urv'

In [54]:
semantic_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In John Wick, the main character seeks revenge on the people who took something he loved from him. It's a story about an ex-hitman who comes out of retirement to track down the gangsters that killed his dog and took everything from him."

# 🤝 Breakout Room Part #2

#### 🏗️ Activity #1

Your task is to evaluate the various Retriever methods against eachother.

You are expected to:

1. Create a "golden dataset"
 - Use Synthetic Data Generation (powered by Ragas, or otherwise) to create this dataset
2. Evaluate each retriever with *retriever specific* Ragas metrics
 - Semantic Chunking is not considered a retriever method and will not be required for marks, but you may find it useful to do a "semantic chunking on" vs. "semantic chunking off" comparision between them
3. Compile these in a list and write a small paragraph about which is best for this particular data and why.

Your analysis should factor in:
  - Cost
  - Latency
  - Performance

> NOTE: This is **NOT** required to be completed in class. Please spend time in your breakout rooms creating a plan before moving on to writing code.

In [51]:
import os
from getpass import getpass
#!pip install -qU ragas==0.2.10
os.environ["RAGAS_APP_TOKEN"] = getpass("Please enter your Ragas API key!")

##### HINTS:

- LangSmith provides detailed information about latency and cost.

In [55]:
import ragas.metrics
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset import TestsetGenerator
from ragas import evaluate, EvaluationDataset, RunConfig
from ragas.metrics import (ContextPrecision, ContextRecall, ContextEntityRecall,  Faithfulness, NoiseSensitivity, ResponseRelevancy)

generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(documents,testset_size=10)





Applying SummaryExtractor:   0%|          | 0/44 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/100 [00:00<?, ?it/s]

Node cbfae9e1-6640-473c-8613-65ef7a7095d2 does not have a summary. Skipping filtering.
Node b181bc37-126d-4e27-b13d-9e803825391d does not have a summary. Skipping filtering.
Node d32809bd-bd98-472e-890a-0e2a0dd0024b does not have a summary. Skipping filtering.
Node 859078bc-9307-4237-984a-40362d6c917d does not have a summary. Skipping filtering.
Node 6e0a16dc-1439-40d3-9558-d3597035d9d8 does not have a summary. Skipping filtering.
Node 4b794e0f-43d5-4485-884b-d93ed7b8934e does not have a summary. Skipping filtering.
Node c5e5b3b2-a434-4dba-9e3e-ddb139c3c7bb does not have a summary. Skipping filtering.
Node 82c934f3-a936-48c8-a62d-55404bfdefa1 does not have a summary. Skipping filtering.
Node a0b28f79-9d21-4626-920d-262b0cc4dde4 does not have a summary. Skipping filtering.
Node 5fb36ed7-34fe-4446-ad57-9586aeca13af does not have a summary. Skipping filtering.
Node 2d3b368e-26a3-477f-b73c-873ced5e78ee does not have a summary. Skipping filtering.
Node dd5cf7c8-35b7-4a86-adc6-52ecdebd098c d

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/244 [00:00<?, ?it/s]

Applying OverlapScoreBuilder:   0%|          | 0/1 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/10 [00:00<?, ?it/s]

In [56]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What Liam Neeson got to do with John Wick movie?,[: 0\nReview: The best way I can describe John...,The review describes John Wick as similar to T...,single_hop_specifc_query_synthesizer
1,What is the general reception of the John Wick...,[: 2\nReview: With the fourth installment scor...,The John Wick film series is apparently loved ...,single_hop_specifc_query_synthesizer
2,What makes John Wick special?,[: 3\nReview: John wick has a very simple reve...,John Wick is special because it features real ...,single_hop_specifc_query_synthesizer
3,Wht makes John Wick such an engaging action fi...,[: 4\nReview: Though he no longer has a taste ...,"John Wick is engaging due to its stylized, vis...",single_hop_specifc_query_synthesizer
4,What is the plot of the first John Wick movie?,[: 5\nReview: Ultra-violent first entry with l...,"In the original John Wick (2014), an ex-hit-ma...",single_hop_specifc_query_synthesizer
5,In what ways does John Wick: Chapter 4 improve...,[<1-hop>\n\n: 19\nReview: John Wick: Chapter 4...,John Wick: Chapter 4 improves upon its predece...,multi_hop_specific_query_synthesizer
6,How does 'John Wick: Chapter 2' compare to 'Jo...,[<1-hop>\n\n: 16\nReview: John Wick Chapter 2 ...,'John Wick: Chapter 2' is praised for its over...,multi_hop_specific_query_synthesizer
7,How does the action in 'John Wick' compare to ...,[<1-hop>\n\n: 19\nReview: I really don't under...,"According to the reviews, 'John Wick' is consi...",multi_hop_specific_query_synthesizer
8,How does Ian McShane contribute to the dynamic...,"[<1-hop>\n\n: 9\nReview: At first glance, John...",Ian McShane plays a significant role in the Jo...,multi_hop_specific_query_synthesizer
9,How does John Wick's quest for his stolen car ...,[<1-hop>\n\n: 5\nReview: Iosef's uncle still h...,John Wick's quest for his stolen car begins wh...,multi_hop_specific_query_synthesizer


In [57]:
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
custom_run_config = RunConfig(timeout=360)
def run_chain_on_dataset_and_evaluate_ragas_metrics(dataset, retrieval_chain, chain_name="Retrieval Chain"):
    """
    Takes a Ragas testset (dataset) and a LangChain chain (retrieval_chain),
    runs the chain for each test sample, and updates the response and retrieved_contexts.
    Returns the updated dataset.
    """
    print(f"Running dataset through {chain_name}...")

    for test_row in dataset:
        chain_output = retrieval_chain.invoke({"question": test_row.eval_sample.user_input})
        # Store only the text content (string) in the response
        test_row.eval_sample.response = chain_output["response"].content
        # Store the page_content of each retrieved document in retrieved_contexts
        test_row.eval_sample.retrieved_contexts = [
            doc.page_content for doc in chain_output["context"]
        ]
    dataset.to_pandas()
    evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())
    result = evaluate(
      dataset=evaluation_dataset,
      metrics=[ContextPrecision(), ContextRecall(), ContextEntityRecall(), Faithfulness(), NoiseSensitivity(), ResponseRelevancy()],
      llm=evaluator_llm,
      run_config=custom_run_config
    )
    return result

chains = [
    ("Naive RAG Chain", naive_retrieval_chain),
    ("BM25 Retriever", bm25_retrieval_chain),
    ("Contextual Compression (Rerank)", contextual_compression_retrieval_chain),
    ("Multi-Query Retriever", multi_query_retrieval_chain),
    ("Parent Document Retriever", parent_document_retrieval_chain),
    ("Ensemble Retriever", ensemble_retrieval_chain),
]



# Run each chain on a fresh copy of 'dataset'
for chain_name, chain in chains:
    # Evaluate the chain with Ragas
    metrics_result = run_chain_on_dataset_and_evaluate_ragas_metrics(
        dataset,
        chain,
        chain_name=chain_name
    )
    
    # Print the results
    print(f"\n=== {chain_name} Metrics ===")
    print(metrics_result)
    print("-"*50)







Running dataset through Naive RAG Chain...


Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

Exception raised in Job[28]: TimeoutError()
Exception raised in Job[58]: TimeoutError()



=== Naive RAG Chain Metrics ===
{'context_precision': 0.8300, 'context_recall': 0.9667, 'context_entity_recall': 0.6667, 'faithfulness': 0.8258, 'noise_sensitivity_relevant': 0.2450, 'answer_relevancy': 0.9579}
--------------------------------------------------
Running dataset through BM25 Retriever...


Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]


=== Parent Document Retriever Metrics ===
{'context_precision': 0.8500, 'context_recall': 0.5317, 'context_entity_recall': 0.5667, 'faithfulness': 0.7386, 'noise_sensitivity_relevant': 0.2712, 'answer_relevancy': 0.8579}
--------------------------------------------------
Running dataset through Ensemble Retriever...


Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

Exception raised in Job[16]: TimeoutError()
Exception raised in Job[28]: TimeoutError()
Exception raised in Job[34]: TimeoutError()
Exception raised in Job[40]: TimeoutError()
Exception raised in Job[46]: TimeoutError()
Exception raised in Job[52]: TimeoutError()
Exception raised in Job[58]: TimeoutError()



=== Ensemble Retriever Metrics ===
{'context_precision': 0.8476, 'context_recall': 1.0000, 'context_entity_recall': 0.6500, 'faithfulness': 0.8382, 'noise_sensitivity_relevant': 0.2500, 'answer_relevancy': 0.9742}
--------------------------------------------------


#### Answer

**Retrieval Methods Comparison**

**Ensemble Retriever**

Performance: Highest overall (precision: 0.85, recall: 1.0, relevancy: 0.97)
Cost: Very high (uses multiple retrievers)
Latency: Slowest (multiple timeouts during evaluation)


**Multi-Query Retriever**

Performance: Excellent recall (1.0) with good precision (0.74)
Cost: High (multiple embedding operations)
Latency: Slow (multiple timeouts)


**Contextual Compression (Rerank)**

Performance: Best precision (0.87) with good recall (0.83)
Cost: Medium-high (embedding + reranking API)
Latency: Moderate


**Naive RAG Chain**

Performance: Strong overall (precision: 0.83, recall: 0.97)
Cost: Low (single embedding operation)
Latency: Moderate (some timeouts)


**Parent Document Retriever**

Performance: Good precision (0.85) but poor recall (0.53)
Cost: Medium (additional processing)
Latency: Fast (no timeouts)


**BM25 Retriever**

Performance: Lowest overall (precision: 0.66, recall: 0.63)
Cost: Very low (no embeddings)
Latency: Very fast



**Analysis**
For the John Wick movie reviews dataset, the Contextual Compression (Rerank) method offers the best balance of performance, cost, and latency. While the Ensemble Retriever achieved the highest overall metrics, its excessive cost and latency make it impractical for most applications. The Contextual Compression approach delivers the highest precision (0.87) with good recall (0.83), making it particularly effective for movie reviews where opinion nuances matter. It strikes an optimal balance by first retrieving candidates efficiently, then applying targeted reranking to improve relevance, resulting in moderate latency without the timeout issues plaguing more complex methods. For budget-conscious implementations, the Naive RAG Chain provides a compelling alternative with strong performance at lower cost, while applications requiring minimal latency might consider BM25 despite its performance trade-offs.